In [1]:
import tensorflow as tf
from scipy import misc
import cv2
import matplotlib.pyplot as plt
import numpy as np
import argparse
import facenet
import detect_face
import os
from os.path import join as pjoin
import sys
import time
import copy
import math
import pickle
from sklearn.svm import SVC
from sklearn.externals import joblib

/home/sohaib/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from pathlib import Path
import json
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects
import pdb
from pathlib import Path

In [9]:
import pdb

In [6]:
PATH = Path('/home/sohaib/Desktop/Tools/real-time-deep-face-recognition/models')
model_files = list(PATH.iterdir())
lenm = len(model_files)

In [8]:
while len(list(PATH.iterdir())) != lenm:
    print('same length')
    classifier_filename = '/home/sohaib/Desktop/Tools/real-time-deep-face-recognition/models/med_soh_kev.pkl'
    lenm = len(model_files)
    
print('different length')    

different length


In [22]:
print('Creating networks and loading parameters')
with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
    # sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    sess = tf.Session(config=tf.ConfigProto(device_count={'GPU': 0}))
    with sess.as_default():
        pnet, rnet, onet = detect_face.create_mtcnn(sess, '/home/sohaib/Desktop/Tools/real-time-deep-face-recognition/det_facenet')

        minsize = 20  # minimum size of face
        threshold = [0.6, 0.7, 0.7]  # three steps's threshold
        factor = 0.709  # scale factor
        margin = 44
        frame_interval = 3
        batch_size = 1000
        image_size = 182
        input_image_size = 160

        HumanNames = ['Kevin', 'Med', 'Sohaib', 'Alessa', 'Kaori', 'Nicola']    # train human name
        HumanNames = sorted(HumanNames)

        print('Loading feature extraction model')
        modeldir = '/home/sohaib/Desktop/Tools/real-time-deep-face-recognition/models/20170512-110547/20170512-110547.pb'
        facenet.load_model(modeldir)

        images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
        embedding_size = embeddings.get_shape()[1]

        classifier_filename = '/home/sohaib/Desktop/Tools/real-time-deep-face-recognition/models/six20180522.pkl'
        classifier_filename_exp = os.path.expanduser(classifier_filename)
        with open(classifier_filename_exp, 'rb') as infile:
            (model, class_names) = pickle.load(infile)
            print('load classifier file-> %s' % classifier_filename_exp)

        video_capture = cv2.VideoCapture(0)
        c = 0

        # #video writer
        # fourcc = cv2.VideoWriter_fourcc(*'DIVX')
        # out = cv2.VideoWriter('3F_0726.avi', fourcc, fps=30, frameSize=(640,480))

        print('Start Recognition!')
        prevTime = 0
        while True:

            while len(list(PATH.iterdir())) != lenm:
                print('different length')
                classifier_filename = '/home/sohaib/Desktop/Tools/real-time-deep-face-recognition/models/med_soh_kev.pkl'
                lenm = len(list(PATH.iterdir()))
                
                classifier_filename_exp = os.path.expanduser(classifier_filename)
                with open(classifier_filename_exp, 'rb') as infile:
                    (model, class_names) = pickle.load(infile)
                    print('load classifier file-> %s' % classifier_filename_exp)
                break;
            
            ret, frame = video_capture.read()

            frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)    # resize frame (optional)

            curTime = time.time()    # calc fps
            timeF = frame_interval

            if c % timeF == 0:
                find_results = []

                if frame.ndim == 2:
                    frame = facenet.to_rgb(frame)
                frame = frame[:, :, 0:3]

                bounding_boxes, _ = detect_face.detect_face(frame, minsize, pnet, rnet, onet, threshold, factor)
                nrof_faces = bounding_boxes.shape[0]
                print('Detected_FaceNum: %d' % nrof_faces)

                if nrof_faces > 0:
                    det = bounding_boxes[:, 0:4]
                    img_size = np.asarray(frame.shape)[0:2]

                    cropped = []
                    scaled = []
                    scaled_reshape = []
                    bb = np.zeros((nrof_faces,4), dtype=np.int32)

                    for i in range(nrof_faces):
                        emb_array = np.zeros((1, embedding_size))

                        bb[i][0] = det[i][0]
                        bb[i][1] = det[i][1]
                        bb[i][2] = det[i][2]
                        bb[i][3] = det[i][3]

                        # inner exception
                        if bb[i][0] <= 0 or bb[i][1] <= 0 or bb[i][2] >= len(frame[0]) or bb[i][3] >= len(frame):
                            print('face is inner of range!')
                            continue

                        cropped.append(frame[bb[i][1]:bb[i][3], bb[i][0]:bb[i][2], :])
                        cropped[i] = facenet.flip(cropped[i], False)
                        scaled.append(misc.imresize(cropped[i], (image_size, image_size), interp='bilinear'))
                        scaled[i] = cv2.resize(scaled[i], (input_image_size, input_image_size),
                                               interpolation=cv2.INTER_CUBIC)
                        scaled[i] = facenet.prewhiten(scaled[i])
                        scaled_reshape.append(scaled[i].reshape(-1, input_image_size, input_image_size, 3))
                        feed_dict = {images_placeholder: scaled_reshape[i], phase_train_placeholder: False}
                        emb_array[0, :] = sess.run(embeddings, feed_dict=feed_dict)
                        predictions = model.predict_proba(emb_array)
                        print(predictions)
                

Creating networks and loading parameters
Loading feature extraction model
Model filename: /home/sohaib/Desktop/Tools/real-time-deep-face-recognition/models/20170512-110547/20170512-110547.pb
load classifier file-> /home/sohaib/Desktop/Tools/real-time-deep-face-recognition/models/six20180522.pkl
Start Recognition!
Detected_FaceNum: 1


/home/sohaib/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:100: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


[[0.78973788 0.03214635 0.06845445 0.03746935 0.01475746 0.0574345 ]]
Detected_FaceNum: 1
[[0.75496756 0.03506463 0.07384829 0.05323644 0.01633349 0.06654959]]
Detected_FaceNum: 1
[[0.76955462 0.03381538 0.06779062 0.05731378 0.01763925 0.05388636]]
Detected_FaceNum: 1
[[0.73074699 0.03618651 0.07380819 0.07933111 0.02278519 0.05714201]]
Detected_FaceNum: 1
[[0.71696113 0.03610477 0.07949601 0.08097981 0.02444276 0.06201552]]
Detected_FaceNum: 1
[[0.78374015 0.03127357 0.06967779 0.0426591  0.01382445 0.05882493]]
Detected_FaceNum: 1
[[0.79474753 0.02945885 0.07406867 0.02849763 0.01139234 0.061835  ]]
Detected_FaceNum: 1
[[0.79505762 0.03149036 0.06577746 0.04101351 0.01350597 0.05315509]]
Detected_FaceNum: 1
[[0.77170914 0.03382275 0.07182153 0.04882603 0.01486783 0.05895273]]
Detected_FaceNum: 1
[[0.78310866 0.03217706 0.07093834 0.04308285 0.01261132 0.05808177]]
Detected_FaceNum: 1
[[0.77890871 0.03308961 0.06388608 0.05163472 0.01931336 0.05316752]]
Detected_FaceNum: 1
[[0.786958

KeyboardInterrupt: 

In [ ]:
#check if new weights where added in the folder
while len(list(PATH.iterdir())) != initial_folder_length:
    print('different length')
    classifier_filename = 'last_weights.pkl'
    initial_folder_length = len(list(PATH.iterdir()))

    # load the new weights inside the model
    classifier_filename_exp = os.path.expanduser(classifier_filename)
    with open(classifier_filename_exp, 'rb') as infile:
        (model, class_names) = pickle.load(infile)
        print('load classifier file-> %s' % classifier_filename_exp)
    break;